In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
# sys.path.append('/Users/floorbroekgaarden/Projects/BHNS_project/Scripts')
# import string

# import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
# import coencodeVarious        as CV


# some often used post-processing functions. 
# from PostProcessingScripts import * 
# import PostProcessingScripts as PP  

import pandas as pd


from astropy import units as u
from astropy import constants as const



from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


%matplotlib inline
import pesummary
from pesummary.io import read
print(pesummary.__version__)
import h5py
# %config InlineBackend.figure_format = 'retina'

import astropy.cosmology
from astropy import units
import pycbc.conversions
import numpy

In [ ]:
def get_cosmology(cosmology=None, **kwargs):
    r"""Gets an astropy cosmology class.

    Parameters
    ----------
    cosmology : str or astropy.cosmology.FlatLambdaCDM, optional
        The name of the cosmology to use. For the list of options, see
        :py:attr:`astropy.cosmology.parameters.available`. If None, and no
        other keyword arguments are provided, will default to
        :py:attr:`DEFAULT_COSMOLOGY`. If an instance of
        :py:class:`astropy.cosmology.FlatLambdaCDM`, will just return that.
    \**kwargs :
        If any other keyword arguments are provided they will be passed to
        :py:attr:`astropy.cosmology.FlatLambdaCDM` to create a custom
        cosmology.

    Returns
    -------
    astropy.cosmology.FlatLambdaCDM
        The cosmology to use.

    Examples
    --------
    Use the default:

    >>> from pycbc.cosmology import get_cosmology
    >>> get_cosmology()
    FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307,
                  Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV,
                  Ob0=0.0486)

    Use properties measured by WMAP instead:

    >>> get_cosmology("WMAP9")
    FlatLambdaCDM(name="WMAP9", H0=69.3 km / (Mpc s), Om0=0.286, Tcmb0=2.725 K,
                  Neff=3.04, m_nu=[0. 0. 0.] eV, Ob0=0.0463)

    Create your own cosmology (see :py:class:`astropy.cosmology.FlatLambdaCDM`
    for details on the default values used):

    >>> get_cosmology(H0=70., Om0=0.3)
    FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=0 K, Neff=3.04, m_nu=None,
                  Ob0=None)

    """
    if kwargs and cosmology is not None:
        raise ValueError("if providing custom cosmological parameters, do "
                         "not provide a `cosmology` argument")
    if isinstance(cosmology, astropy.cosmology.FlatLambdaCDM):
        # just return
        return cosmology
    if kwargs:
        cosmology = astropy.cosmology.FlatLambdaCDM(**kwargs)
    else:
        if cosmology is None:
            cosmology = DEFAULT_COSMOLOGY
        if cosmology not in astropy.cosmology.parameters.available:
            raise ValueError("unrecognized cosmology {}".format(cosmology))
        cosmology = getattr(astropy.cosmology, cosmology)
    return cosmology


def z_at_value(func, fval, unit, zmax=1000., **kwargs):
    r"""Wrapper around astropy.cosmology.z_at_value to handle numpy arrays.

    Getting a z for a cosmological quantity involves numerically inverting
    ``func``. The ``zmax`` argument sets how large of a z to guess (see
    :py:func:`astropy.cosmology.z_at_value` for details). If a z is larger than
    ``zmax``, this will try a larger zmax up to ``zmax * 10**5``. If that still
    is not large enough, will just return ``numpy.inf``.

    Parameters
    ----------
    func : function or method
        A function that takes redshift as input.
    fval : float
        The value of ``func(z)``.
    unit : astropy.unit
        The unit of ``fval``.
    zmax : float, optional
        The initial maximum search limit for ``z``. Default is 1000.
    \**kwargs :
        All other keyword arguments are passed to
        :py:func:``astropy.cosmology.z_at_value``.

    Returns
    -------
    float
        The redshift at the requested values.
    """
    fval, input_is_array = pycbc.conversions.ensurearray(fval)
    # make sure fval is atleast 1D
    if fval.size == 1 and fval.ndim == 0:
        fval = fval.reshape(1)
    zs = numpy.zeros(fval.shape, dtype=float)  # the output array
    for (ii, val) in enumerate(fval):
        try:
            zs[ii] = astropy.cosmology.z_at_value(func, val*unit, zmax=zmax,
                                                  **kwargs)
        except CosmologyError:
            # we'll get this if the z was larger than zmax; in that case we'll
            # try bumping up zmax later to get a value
            zs[ii] = numpy.inf
    # check if there were any zs > zmax
    replacemask = numpy.isinf(zs)
    # try bumping up zmax to get a result
    if replacemask.any():
        # we'll keep bumping up the maxz until we can get a result
        counter = 0  # to prevent running forever
        while replacemask.any():
            kwargs['zmin'] = zmax
            zmax = 10 * zmax
            idx = numpy.where(replacemask)
            for ii in idx:
                val = fval[ii]
                try:
                    zs[ii] = astropy.cosmology.z_at_value(
                        func, val*unit, zmax=zmax, **kwargs)
                    replacemask[ii] = False
                except CosmologyError:
                    # didn't work, try on next loop
                    pass
            counter += 1
            if counter == 5:
                # give up and warn the user
                logging.warning("One or more values correspond to a "
                                "redshift > {0:.1e}. The redshift for these "
                                "have been set to inf. If you would like "
                                "better precision, call God.".format(zmax))
                break
    return pycbc.conversions.formatreturn(zs, input_is_array)


def _redshift(distance, **kwargs):
    r"""Uses astropy to get redshift from the given luminosity distance.

    Parameters
    ----------
    distance : float
        The luminosity distance, in Mpc.
    \**kwargs :
        All other keyword args are passed to :py:func:`get_cosmology` to
        select a cosmology. If none provided, will use
        :py:attr:`DEFAULT_COSMOLOGY`.

    Returns
    -------
    float :
        The redshift corresponding to the given luminosity distance.
    """
    cosmology = get_cosmology(cosmology='WMAP9')
    return z_at_value(cosmology.luminosity_distance, distance, units.Mpc)

In [1]:
# BHNSsGWTC1 = ['GW150914', 'GW151012', 'GW151226', 'GW170104', 'GW170608', 'GW170729',  'GW170809', 'GW170814', 'GW170818', 'GW170823']
# BNSsGWTC1 =['GW170817']

# BHNSsGWTC2 = ['GW190408_181802','GW190412','GW190413_052954','GW190413_134308','GW190421_213856',\
# 'GW190424_180648','GW190503_185404','GW190512_180714',\
# 'GW190513_205428','GW190514_065416','GW190517_055101','GW190519_153544','GW190521_074359',\
# 'GW190521','GW190527_092055','GW190602_175927','GW190620_030421','GW190630_185205','GW190701_203306',\
# 'GW190706_222641','GW190707_093326','GW190708_232457','GW190720_000836',\
# 'GW190727_060333','GW190728_064510','GW190731_140936','GW190803_022701','GW190828_063405',\
# 'GW190828_065509','GW190910_112807','GW190915_235702','GW190924_021846','GW190929_012149',\
#  'GW190930_133541']
# BNSsGWTC2 =['GW190425']



BHNSsGWTC2 = ['GW190425', 'GW190814', 'GW190426_152155'] 

def obtainGWTC1_MassAndWeight(DCOtype, whichGWTC='ALL'):
    
    
    M1all, M2all = [], []
    Weightsall = []
    
    
    if DCOtype=='BHNS':
        print('doing GWTC-2 BHNS')
        for ind_BHNS, BHNS in enumerate(BHNSsGWTC2):
            print(BHNS)
            BHNS_file = '/Users/floorbroekgaarden/Projects/GWTC-1_LVK_catalog/GWTC-2_sample_release/'+BHNS+'_comoving.h5'
            BHNS = h5.File(BHNS_file, 'r')
            
            data = read(BHNS_file)
            samples_dict = data.samples_dict
            posterior_samples = samples_dict['PublicationSamples']
            parameters = sorted(list(posterior_samples.keys()))

            M1 = posterior_samples['mass_1_source']
            M2 = posterior_samples['mass_2_source']

            

            M1all = np.concatenate((M1, M1all))
            M2all = np.concatenate((M2, M2all))


            w = np.ones_like(M1) * (1./len(M1))
            Weightsall = np.concatenate((w, Weightsall))        

      
    return M1all, M2all, Weightsall

# to test: 
# M1all, M2all, Weightsall  = obtainGWTC1_MassAndWeight(DCOtype='BHBH')

In [2]:

def calculateConfidenceIntervals_GWTC(DCOtype='BHNS', whichGWTC='ALL'): 
    """ 
    Calculate confidence intervals from the GWTC-1 and GWTC-2 catalog (observed)
    input:
    

    """


    # prepare DataFrame 
    xvarHeaders = ['Mass1', 'Mass2', \
                    'TotMass', 'ChirpMass', 'q']

    xvarUnits = ['Msun', 'Msun', 'Msun', 'Msun', '#']

    # quantiles that I want to know
    y_quantiles  =          [0.005,   0.05,   0.16,   0.25,   0.5,   0.75,   0.84,   0.95,  0.995]
    indexnames   = ['unit', '0.005', '0.05', '0.16', '0.25', '0.5', '0.75', '0.84', '0.95', '0.995']
    # nr of rows and columns that will be used:
    ncol_var = len(xvarHeaders)   
    ncol_Rate_det = 1

    nrows = len(y_quantiles) + 1 # +1 for units (see below)
    # store variables, and Observed and intrinsic rates for all MSSFR variations:
    ncol = ncol_var #* (ncol_MSSFR) # for each MSSFR I want to give quantiles for each xparam 
    df_placeholder = np.zeros((nrows, ncol)) # will be filled in loop: 

    headernames=[]
    units=[]
    for ind_s, ss in enumerate(xvarHeaders):
        sss = ss 
        headernames.append(sss)
        units.append(xvarUnits[ind_s])

    # store dataFrame with zeros that we will fill on the go:
    dfw = pd.DataFrame(data=df_placeholder, columns=headernames, index=indexnames)   
    # add units at first row (index=0)
    dfw.iloc[0]=units        
        
                

    # obtain BH and NS masses from combining GW posteriors.
    # by definition LIGO gives Most massive and least massive. 
    MBH, MNS, weights  = obtainGWTC1_MassAndWeight(DCOtype=DCOtype, whichGWTC=whichGWTC)
    xvarlist = [MBH, MNS, (MBH+MNS), chirpmass(MBH, MNS), (MBH/MNS)]


    # calculate quantiles with bootstrapping (if Nrepeats>1)
    for ind_xvar, xvar in enumerate(xvarlist):

        Nrepeats=1
        boot_xvar = np.asarray(xvar)
        boot_index = np.arange(len(boot_xvar))

        bootstrap_array = np.zeros((Nrepeats, len(y_quantiles)))

        for ind_r, repeat in enumerate(range(Nrepeats)):
            # (bootstrap) re-sample a random set of samples with replacement from existing samples
            # do this by drawing random sample indecis nrs, each nr corresponds to a sample  
            boot_randindex = np.random.choice(boot_index, size=len(boot_index), replace=True, p=None)
            boot_randweight   = np.asarray(weights)[boot_randindex] # get the corresponding weights 
            boot_randsamples   = np.asarray(boot_xvar)[boot_randindex] # get the corresponding samples


            xvar_quantiles = weighted_quantile(values=boot_randsamples, quantiles=y_quantiles, \
                 sample_weight=boot_randweight)
            bootstrap_array[ind_r] = xvar_quantiles


            xvar_quantiles_bootstrapped = np.mean(bootstrap_array, axis=0)

            dfw_key = xvarHeaders[ind_xvar] 
            dfw[dfw_key][1:] = xvar_quantiles_bootstrapped

        
    if whichGWTC=='GWTC-1':
        dfCSVname = '/Users/floorbroekgaarden/Projects/BHNS_project/PlottingScripts/9_PredictedDistributions_BPS_and_MSSFR_variations/csvFilesConfidenceIntervals/ConfidenceIntervals_' + 'GW170817'+ '_' + DCOtype + '.csv' 

    elif whichGWTC=='GWTC-2':
        dfCSVname = '/Users/floorbroekgaarden/Projects/BHNS_project/PlottingScripts/9_PredictedDistributions_BPS_and_MSSFR_variations/csvFilesConfidenceIntervals/ConfidenceIntervals_' + 'GW190425'+ '_' + DCOtype + '.csv' 

    else:
        dfCSVname = '/Users/floorbroekgaarden/Projects/BHNS_project/PlottingScripts/9_PredictedDistributions_BPS_and_MSSFR_variations/csvFilesConfidenceIntervals/ConfidenceIntervals_' + 'GWTCs'+ '_' + DCOtype + '.csv' 
    dfw.to_csv(dfCSVname) 
    
    
    print()
    print('finished')


    
    return


In [ ]:
calculateConfidenceIntervals_GWTC(DCOtype='BHBH', whichGWTC='All')
calculateConfidenceIntervals_GWTC(DCOtype='BHNS')

In [ ]:
DCOtype='BHNS'
dfname ='/Users/floorbroekgaarden/Projects/BHNS_project/PlottingScripts/9_PredictedDistributions_BPS_and_MSSFR_variations/csvFilesConfidenceIntervals/ConfidenceIntervals_' + 'GWTCs'+ '_' + DCOtype + '.csv'
df = pd.read_csv(dfname, header=[0], index_col=[0], skiprows=[1])
print(df)

ind_GWTC = 0

q0_005= (df.iloc[0][ind_GWTC])#[ind_mssfr] #
q0_05 = (df.iloc[1][ind_GWTC])#[ind_mssfr]
q0_25 = (df.iloc[3][ind_GWTC])
q0_5 = (df.iloc[4][ind_GWTC])
q0_75 = (df.iloc[5][ind_GWTC])
q0_95 = (df.iloc[7][ind_GWTC])
q0_995=(df.iloc[8][ind_GWTC])

plt.scatter(q0_005, q0_005)
plt.show()